# Validation of two-dimensional video-based inference of finger

## OpenPose video to keypoints
Run this notebook to obtain coordinates of the hands for a video input.


## Resources
- OpenPose documentation: https://cmu-perceptual-computing-lab.github.io/openpose/html/index.html

In [ ]:
import os
from os.path import exists, join, basename, splitext
import json
import numpy as np

In [ ]:
git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
  # install new CMake becaue of CUDA10
  !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
  !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
  # clone openpose
  !git clone -q --depth 1 $git_repo_url
  !sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
  # install system dependencies
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  # install python dependencies
  !pip install -q youtube-dl
  # build openpose
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`

## Detect poses on video
Outputs: video with hand overlays in .mp4 format, .zip file of keypoint coordinates.

In [ ]:
# detect poses 
#!rm openpose.avi
!cd openpose && ./build/examples/openpose/openpose.bin --video ../Part1_QTM_R_CENTER.avi --render_pose 0 --hand --hand_render 2 --write_json ./output_RC_P1/ --display 0  --write_video ../openpose_RC_P1.avi
# convert the result into MP4
!ffmpeg -y -loglevel info -i openpose_RC_P1.avi output_RC_P1.mp4
# save json files to zip
!zip -r -q keypoints_RC_P1.zip openpose/output_RC_P1

Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total time: 728.475812 seconds.
ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-lib

In [ ]:
# detect poses 
#!rm openpose.avi
!cd openpose && ./build/examples/openpose/openpose.bin --video ../Part1_QTM_L_CENTER.avi --render_pose 0 --hand --hand_render 2 --write_json ./output_LC_P1/ --display 0  --write_video ../openpose_LC_P1.avi
# convert the result into MP4
!ffmpeg -y -loglevel info -i openpose_LC_P1.avi output_LC_P1.mp4
# save json files to zip
!zip -r -q keypoints_LC_P1.zip openpose/output_LC_P1

Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total time: 683.903610 seconds.
ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-lib

# Reformat keypoint coordinates from Json to Text
Output: .zip files containing coordinates in .txt format.

In [ ]:
!mkdir keypoints_txt
!mkdir keypoints_txt/body
!mkdir keypoints_txt/right_hand
!mkdir keypoints_txt/left_hand
root = "/content/openpose/output_RC_P1/"
file_name_prefix = "Part1_QTM_R_CENTER_"
n_frames_RC = 8606
def reformat_coordinates(root, file_name):
    with open(root + file_name+".json") as f:
        data = json.load(f)
    body = data['people'][0]['pose_keypoints_2d']
    right_hand = data['people'][0]['hand_right_keypoints_2d']
    left_hand = data['people'][0]['hand_left_keypoints_2d']
    body_x = body[::3]
    body_y = body[1::3]
    right_hand_x = right_hand[::3]
    right_hand_y = right_hand[1::3]
    left_hand_x = left_hand[::3]
    left_hand_y = left_hand[1::3]
    body_txt = np.array([body_x, body_y]).T.tolist()
    right_hand_txt = np.array([right_hand_x, right_hand_y]).T.tolist()
    left_hand_txt = np.array([left_hand_x, left_hand_y]).T.tolist()
    np.savetxt("keypoints_txt/body/"+"out_body_"+file_name+".txt", body_txt)
    np.savetxt("keypoints_txt/right_hand/"+"right_hand_"+file_name+".txt", right_hand_txt)
    np.savetxt("keypoints_txt/left_hand/"+"left_hand_"+file_name+".txt", left_hand_txt)
for n in range(n_frames_RC):
  if n < 10:
      file_name = file_name_prefix + "00000000000" + str(n) + '_keypoints' 
  if (n >= 10) & (n < 100):
      file_name = file_name_prefix + "0000000000" + str(n) + '_keypoints'
  if (n >= 100) & (n < 1000):
      file_name = file_name_prefix + "000000000" + str(n) + '_keypoints'
  elif (n >= 1000) & (n < 10000):
      file_name = file_name_prefix + "00000000" + str(n) + '_keypoints'
  reformat_coordinates(root, file_name)
!zip -r -q keypoints_RC_P1_txt.zip keypoints_txt

# Rename **keypoints_txt** to a different name before the next step to avoid writing to the same folder.

In [ ]:
!mkdir keypoints_txt
!mkdir keypoints_txt/body
!mkdir keypoints_txt/right_hand
!mkdir keypoints_txt/left_hand
root = "/content/openpose/output_LC_P1/"
file_name_prefix = "Part1_QTM_L_CENTER_"
n_frames_LC = 8123

for n in range(n_frames_LC):
  if n < 10:
      file_name = file_name_prefix + "00000000000" + str(n) + '_keypoints' 
  if (n >= 10) & (n < 100):
      file_name = file_name_prefix + "0000000000" + str(n) + '_keypoints'
  if (n >= 100) & (n < 1000):
      file_name = file_name_prefix + "000000000" + str(n) + '_keypoints'
  elif (n >= 1000) & (n < 10000):
      file_name = file_name_prefix + "00000000" + str(n) + '_keypoints'
  reformat_coordinates(root, file_name)
!zip -r -q keypoints_LC_P1_txt.zip keypoints_txt